In [17]:
import pathlib
import os

import pandas as pd

from translations.age_translations import EUROSTAT_AGES_CONVERSION
from translations.nuts3_translations import EU_DECODE_NUTS3_REGIONS
from translations.sex_translations import EUROSTAT_SEX_CONVERSION
from column_bindings.eurostat_columns import *

In [18]:
raw_mortality_eurostat_file = pathlib.Path(f'{os.getcwd()}/../../raw_data/demo_r_mweek3.tsv')
df = pd.read_csv(raw_mortality_eurostat_file, sep='\t')


In [21]:
# Drop mortality for years different than 2015-2022
drop_cols = [col for col in df.columns[1:] if col[:4] not in [str(year) for year in range(2015, 2023)]]
df.drop(drop_cols, axis=1, inplace=True)

# Drop countries different than Bulgaria (BG)
bg_index = df[df[OriginalFileBindings.all_demography].str.contains("BG")].index
df = df.loc[bg_index, :]

In [22]:
# Convert year_week columns containing mortality to a single column with mortality contained in a single other
df = df.melt(id_vars=OriginalFileBindings.all_demography, 
             var_name=f'{EuroStatColumns.year}_{EuroStatColumns.week}', 
             value_name=EuroStatColumns.mortality)

In [23]:
# Convert files header from comma separated demograpic data in a single column to separate columns with each respective demographic data point
expand_columns = [OriginalFileBindings.units, 
                  EuroStatColumns.sex,
                  EuroStatColumns.age, 
                  EuroStatColumns.location]

df[expand_columns] = df[OriginalFileBindings.all_demography].str.split(',', expand=True)
df.drop([OriginalFileBindings.all_demography, OriginalFileBindings.units], axis=1, inplace=True)

In [24]:
# Split year_week column to distinct columns. Drop original column. Strip unnecessary white spaces for week.
df[[EuroStatColumns.year, EuroStatColumns.week]] = df[f'{EuroStatColumns.year}_{EuroStatColumns.week}'].str.split('W', expand=True)
df.drop([f'{EuroStatColumns.year}_{EuroStatColumns.week}'], axis=1, inplace=True)
df[EuroStatColumns.week] = df[EuroStatColumns.week].str.strip()

In [25]:
# Remove missing, expected and preliminary markings from mortality data
df[EuroStatColumns.mortality] = df[EuroStatColumns.mortality].str.replace(':' , '0')
df[EuroStatColumns.mortality] = df[EuroStatColumns.mortality].str.replace(' p' , '')
df[EuroStatColumns.mortality] = df[EuroStatColumns.mortality].str.replace(' e' , '')
df[EuroStatColumns.mortality]  = df[EuroStatColumns.mortality].map(int)

In [26]:
# Standardize age and sex between files
df[EuroStatColumns.age] = df[EuroStatColumns.age].apply(lambda x: EUROSTAT_AGES_CONVERSION.get(x))
df[EuroStatColumns.sex] = df[EuroStatColumns.sex].apply(lambda x: EUROSTAT_SEX_CONVERSION.get(x))

In [27]:
# Convert Table so each year is a separate column.
df = df.pivot_table(columns=[EuroStatColumns.year], 
                    index=[EuroStatColumns.sex, 
                           EuroStatColumns.age,
                           EuroStatColumns.location,
                           EuroStatColumns.week], 
                    values='Mortality').reset_index()

In [28]:
# filter out unknown regions (e.g. BGX, BGXX) or non-NUTS3 regions like (e.g. BG)
nuts_3_regions = [item for item in df.Location.unique() if len(item)==5 and 'BGX' not in item]
nuts_3_mask = df[df[EuroStatColumns.location].isin(nuts_3_regions)].index
df = df.loc[nuts_3_mask, :]
df[EuroStatColumns.location] = df[EuroStatColumns.location].apply(lambda x: EU_DECODE_NUTS3_REGIONS.get(x))

In [29]:
# filter out unknown weeks (e.g. week 99)
week_mask = [str(week).zfill(2) for week in range(1, 54)]
df = df[df[EuroStatColumns.week].isin(week_mask)]

In [34]:
save_location = raw_mortality_eurostat_file = pathlib.Path(f'{os.getcwd()}/../../cleaned_data/Eurostat_mortality_2015_2022(Bulgaria).csv')
df.to_csv(save_location, index=False)